<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

# Objetivos de la Unidad

- Conocer cómo se realizan las tareas en un sistema multitarea
- Conocer los conceptos de proceso y de _thread_
- Conocer y usar las bibliotecas de clases que provee Python para el manejo de threads
- Aplicar OOP utilizando ideas de _multithreading_ y concurrencia

## Threading

Los **_threads_** (o hilos) son las unidades más pequeñas que pueden ser programadas para ser ejecutadas en un sistema operativo. Estos permiten que varios procesos livianos puedan ser ejecutados simultáneamente dentro de un mismo programa o proceso mayor. Los _threads_ que están dentro de un mismo proceso comparten la memoria y estado de las variables del proceso. Este uso compartido de los recursos le permite a los _threads_ ejecutarse más rápido que correr varias instancias del mismo programa.

Cada proceso tiene al menos un _thread_, el que corresponde al mismo proceso en ejecución. Cuando un proceso crea varios _threads_, estos son ejecutados como procesos _paralelos_. En una máquina con un sólo procesador, este _paralelismo_ es aproximado mediante _thread scheduling_ o _time slicing_; esto es, cada un número determinado de instrucciones, el hilo en ejecución es detenido y otro hilo en la cola de espera es ejecutado durante el mismo número de instrucciones. Esta alternancia entre procesos, ejecutados a la velocidad del procesador, brinda la sensación de paralelismo. En este caso, no existe un verdadero aumento de la velocidad de ejecución, pero sí de la manera en que el programa responde. Sin embargo, en máquinas con más procesadores, efectivamente se logra la ejecución en paralelo de varios _threads_ que tiene como resultado una ejecución más rápida del programa. La siguiente figura muestra un esquema de la interacción de los _threads_ con el proceso principal.

![thread](imgs/thread.png)

Algunos ejemplos, donde resulta útil implementar _threads_, a pesar del uso de _time slicing_ en computadores con un único procesador, son:

- Interfaces de rápida respuesta donde se necesita interactuar con el usuario mientras se ejecuta algún proceso de cómputo pesado. Por ejemplo, las interfaces gráficas en un computador.
- Delegación de trabajos que siguen el patrón _consumidor-productor_, en donde existen procesos que se ejecutan secuencialmente, pero que son independientes entre ellos. Como por ejemplo, un _thread_ que se encargue de poner los _frames_ capturados desde una cámara de video en una cola, y otro _thread_ que procese estos cuadros y los saque de la cola.
- Aplicaciones multiusuarios, en donde cada _thread_ se encargaría de las peticiones de cada usuario independientemente. Por ejemplo, sistemas operativos multiusuario.

### Creación de _threads_

En Python 3, el manejo de _threads_ se realiza mediante la librería `threading`, en donde los _threads_ son representados por objetos. Muchas de las funcionalidades necesarias están implementadas en esta librería. La clase `Thread` crea un nuevo hilo. Esta requiere como argumento básico, entregado a través de `target`, la función objetivo que debe ser ejecutada durante el _thread_. Cada objeto `thread` es ejecutado mediante el método `start()`. La clase `Thread` también admite mediante el argumento `name`, que es el nombre que le queremos dar al hilo. En el siguiente ejemplo, crearemos tres _threads_ `t1`, `w1`, y `w2`, que ejecutarán distintas instancias de las funciones `service` y `worker`.

In [1]:
import threading
import time


def worker():
    print("{} starting...".format(threading.currentThread().getName()))
    # Pondremos a dormir el thread por t segundos simulando 
    # que ocurre algun proceso dentro de la función
    time.sleep(2) 
    print("{} exiting...\n".format(threading.currentThread().getName()))

def service():
    print("{} starting...".format(threading.currentThread().getName()))
    # Ponemos a dormir el thread por 4 segundos simulando
    # un proceso dentro de la función
    time.sleep(4) 
    print("{} exiting...\n".format(threading.currentThread().getName()))

# Creamos los threads usando la clase Thread
t1 = threading.Thread(name='Thread 1', target=service)
w1 = threading.Thread(name='Thread 2', target=worker)
w2 = threading.Thread(target=worker) # En este caso usa el nombre asignado por defecto

# Se inicializan los threads creados
w1.start()
w2.start()
t1.start()

# Todas estas líneas serán ejecutadas mientras los threads se ejecutan independientemente del programa
# principal
print('Fueron creados 3 threads')

for i in range(10):
    print(i)

Fueron creados 3 threadsThread 2 starting...Thread-6 starting...Thread 1 starting...


0

1
2
3
4
5
6
7
8
9


En el ejemplo, vemos que una vez inicializados los _threads_, el programa principal continúa con el resto de instrucciones mientras los _threads_ se ejecutan. Luego los tres _threads_ terminan independientemente en distintos momentos. El programa principal esperará a que todos los _threads_ hayan terminado correctamente.

También es posible, mediante el atributo o llave `args`, pasar los argumentos a la función definida como `target` del _thread_.

In [2]:
import threading
import time

def worker(tiempo):
    print("{} starting with time {}".format(threading.currentThread().getName(), tiempo))
    time.sleep(tiempo) # pone a dormir el thread por t segundos simulando que ocurre algun proceso dentro de la función
    print("{} exiting...\n".format(threading.currentThread().getName()))

    
# Se crean los threads usando la clase Thread, asociada a la función objetivo para 
# ser ejecutada por el thread, y los atributos de la función son ingresados mediante la llave args. 
# En este ejemplo, args tiene asignado una tupla de un solo elemento. Por tal razóni
# debe ser escrita como (valor,)

w = threading.Thread(name='Thread 2', target=worker, args=(3,))
w.start()
w.join()

Thread 2 starting with time 3
Thread 2 exiting...

Thread-6 exiting...

Thread 2 exiting...



En concordancia con el modelamiento basado en OOP, podemos crear también un _thread_ heredando desde la clase `Thread` y redefiniendo el método `run()`, el cual se ejecuta luego de hacer `Thread.start()`.

In [3]:
import threading
import time


class Worker(threading.Thread):
    # Este será nuestro nuevo Worker basado en Thread
    def __init__(self, time):
        super().__init__()
        self.time = time
    
    def run(self):
        # Este metodo ejecuta el proceso de este thread
        # cuando lo iniciamos mediante el metodo start()
        print("{} starting...\n".format(threading.currentThread().getName()))
        
        # Pondremos a dormir el thread por t segundos
        # simulando que ocurre algun proceso dentro de la función
        t0 = time.time()
        time.sleep(self.time) 
        print("{} exiting after {} sec.\n".format(threading.currentThread().getName(), time.time() - t0))

        
class Service(threading.Thread):
    # Este sera el nuevo Thread service basado en Thread
    def __init__(self, time):
        super().__init__()
        self.time = time
    
    def run(self):
        print("{} starting...\n".format(threading.currentThread().getName()))
        # El thread dormira por 4 segundos simulando un proceso dentro de la función
        t0 = time.time()
        time.sleep(self.time) 
        print("{} exiting after {} sec.\n".format(threading.currentThread().getName(), time.time() - t0))
        

# Se crean los threads usando la clase Thread
t1 = Service(3)
w1 = Worker(4)
w2 = Worker(5)

# Se inicializan los threads creados
w1.start()
w2.start()
t1.start()

# Todas estas líneas serán ejecutadas mientras los threads se ejecutan independientemente del programa principal.
print('Fueron creados 3 threads')

for i in range(10):
    print(i)

Thread-8 starting...
Thread-9 starting...
Thread-7 starting...
Fueron creados 3 threads



0
1
2
3
4
5
6
7
8
9


### `join()`

Si necesitamos que el programa principal espere que termine la ejecución de algún _thread_ (o grupo de _threads_), debemos usar del método `join(<tiempo-máximo-espera>)` después de que el _thread_ ha sido iniciado mediante `start()`. Al usar `join()`, el programa principal queda bloqueado hasta que los _threads_ referenciados terminen correctamente. Si el tiempo máximo de espera no ha sido definido, el programa principal esperará indefinidamente hasta que el thread referenciado termine. La figura siguiente explica gráficamente la ejecución del programa usando `join()`.

![join](imgs/join.png)

Veamos ahora el mismo ejemplo anterior pero con la incorporación de `join()`.

In [4]:
import threading
import time

# Usamos la definicion de los Thread declarados en el ejempo anterior

# Se crean los threads usando la clase Thread
t1 = Service(1)
w1 = Worker(6)
w2 = Worker(4)

# Se inicializan los threads creados
t1.start()
w1.start()
w2.start()

# Aquí incorporamos el método join() para bloquear el programa principal
t1.join()
w1.join()
w2.join()


# Una vez que los threads son creados el programa principal sigue su curso y espera 
# a que los threads terminen. Por ejemplo, todas estas líneas serán ejecutadas después 
# que los threads sean ejecutados, pero el programa principal no se cerrará hasta que 
# los threas hayan completado su ejecución.
print('Fueron creados 3 threads')

for i in range(10):
    print(i)

Thread-10 starting...
Thread-11 starting...
Thread-12 starting...



Thread 1 exiting...

Thread-10 exiting after 1.0016515254974365 sec.

Thread-7 exiting after 3.0004642009735107 sec.

Thread-8 exiting after 4.0012640953063965 sec.

Thread-12 exiting after 4.001278877258301 sec.

Thread-9 exiting after 5.000436782836914 sec.

Thread-11 exiting after 6.000386476516724 sec.
Fueron creados 3 threads

0
1
2
3
4
5
6
7
8
9


### `isAlive()`

Es posible identificar si un _thread_ ha terminado su ejecución mediante el uso del método `isAlive()`. Por lo general este método se implementa para saber el estado del _thread_ después del uso de `join()`, al cual se le ha definido su tiempo máximo de espera. Usando las funciones `service` y `worker` podemos hacer lo siguiente.

In [5]:
import threading
import time


# Se crean los threads usando la clase Thread
t = Service(2)#threading.Thread(target=service)

# Se inicializan el threads
t.start()
t.join(5) # el programa esperará por 5 segundos a que el thread termine

# Podemos comprobar si el thread terminó
if not t.isAlive():
    print('El thread ha terminado correctamente')


Thread-13 starting...

Thread-13 exiting after 2.0009405612945557 sec.
El thread ha terminado correctamente



Podemos evitar el uso de los `print` con la librería `logging`, embebiendo el nombre de cada _thread_ en su _log message_ usando el código `%(threadName)s`. Por ejemplo,

In [6]:
import logging
import imp

# Estas dos lineas son necesarias sólo por el ipython notebook
imp.reload(logging)
logging.basicConfig(format='[%(levelname)s] (%(threadName)-10s) %(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

# Con esta línea basta si se ejecuta desde otro IDE
# logging.basicConfig(level = logging.DEBUG, format='[%(levelname)s] (%(threadName)-10s) %(message)s', )


def worker():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

    
def service():
    logging.debug('Starting')
    time.sleep(4)
    logging.debug('Exiting')


t1 = threading.Thread(name='service', target=service)
w1 = threading.Thread(name='worker', target=worker)
w2 = threading.Thread(target=worker)

w1.start()
w2.start()
t1.start()


[DEBUG] (worker    ) Starting
[DEBUG] (Thread-14 ) Starting
[DEBUG] (service   ) Starting


## _Daemons_

En el ejemplo anterior, el programa espera a que todos los _threads_ terminen su trabajo antes de terminar su ejecución. Los **_daemon threads_** son los que a pesar de que estén aún corriendo, no impiden que el programa principal termine. En general, los _daemons_ tienen una menor prioridad en el orden de ejecución de los _threads_. 

Si no existieran los _daemon threads_, tendríamos que preocuparnos de tener en cuenta todos nuestros _threads_ y preocuparnos de terminarlos a todos para que el programa principal pueda terminar, con los _daemon threads_ podemos ponerlos a correr y olvidarnos de ellos, y cuando el programa termina, los _daemon threads_ serán terminados inmediatamente. El programa principal terminará cuando todos los _threads no-daemon_ hayan finalizado.


In [7]:
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )


def daemon():
    logging.debug('Starting')
    time.sleep(4)
    logging.debug('Exiting')

d1 = threading.Thread(name='daemon', target=daemon)
d1.setDaemon(True)#aquí decimos que el thread es efectivamente un daemon thread

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t1 = threading.Thread(name='non-daemon', target=non_daemon)

d1.start()
t1.start()

# ojo que la última línea del output no se genera en un IDE tradicional, 
# sólo el ipython notebook lo ejecuta de todas maneras

[DEBUG] (daemon    ) Starting
[DEBUG] (non-daemon) Starting
[DEBUG] (non-daemon) Exiting


En el _output_ anterior podemos apreciar que el programa principal termina antes que el _daemon thread_. Es posible hacer que el programa espere a un _daemon thread_ mediante el uso de `join()`, lo que tendría sentido cuando debemos esperar algún resultado del _thread_ para el resto del programa.

Más claramente, veamos la salida del mismo programa donde utilizamos `join()` para esperar a que los _threads_ completaran su misión.

In [8]:
import threading
import time


def worker():
    print("{} starting...".format(threading.currentThread().getName()))
    time.sleep(2) # pone a dormir el thread por t segundos simulando que ocurre algun proceso dentro de la función
    print("{} exiting...".format(threading.currentThread().getName()))

    
def service():
    print("{} starting...".format(threading.currentThread().getName()))
    time.sleep(4) # pone a dormir el thread por 4 segundos simulando un proceso dentro de la función
    print("{} exiting...".format(threading.currentThread().getName()))


# Se crean los threads usando la clase Thread
t1 = threading.Thread(name='Thread 1', target=service)
w1 = threading.Thread(name='Thread 2', target=worker)

# Se configuran los threads como Daemon
w1.setDaemon(True)
t1.setDaemon(True)

# Se inicializan los threads
w1.start()
t1.start()

Thread 2 starting...Thread 1 starting...



La siguiente figura muestra la salida en la consola para el programa **con** los _threads_ configurados como `Daemon`. Los _threads_ se interrumpen abruptamente, luego de que el programa principal termina su ejecución.

![daemon](imgs/daemon_thread.png)

Comparemos el resultado anterior con la salida en la consola para el programa **sin** los _threads_ configurados como `Daemon`, como muestra la imagen siguiente. Podemos ver que la ejecución de los _threads_ fue completada y que el programa no se cierra hasta que los _threads_ terminan.

![non_daemon](imgs/nodaemon_thread.png)

In [9]:
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)


def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

    
t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

d.join()
t.join()

[DEBUG] (daemon    ) Starting
[DEBUG] (non-daemon) Starting
[DEBUG] (non-daemon) Exiting
[DEBUG] (worker    ) Exiting
[DEBUG] (Thread-14 ) Exiting
[DEBUG] (daemon    ) Exiting


Thread 2 exiting...


Podemos ingresar como parámetros a `join` el número máximo de segundos que queremos que espere al _daemon_ para que el programa principal pueda terminar, así después de ese tiempo, `join` retornaría igual.

In [10]:
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

class Daemon(threading.Thread):
    
    def __init__(self):
        super().__init__()
        # Cuando inicializamos el Thread lo declaramos como Daemon
        self.daemon = True
    
    def run(self):
        logging.debug('Starting')
        time.sleep(2)
        logging.debug('Exiting')

d = Daemon()
d.start()

[DEBUG] (Thread-15 ) Starting


In [11]:
logging.basicConfig(level=logging.DEBUG,
                    format='(%(threadName)-10s) %(message)s',
                    )

def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

d = threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

d.join(1)
print('d está vivo?: {}'.format(d.isAlive()))
t.join()

[DEBUG] (daemon    ) Starting
[DEBUG] (non-daemon) Starting
[DEBUG] (non-daemon) Exiting


d está vivo?: True


Al crear un _thread_ como objeto podemos indicar si este es `Daemon` durante su inicialización usando el atributo `daemon`. Una vez que se ha llamado el método `start()` un _thread_ no puede cambiar desde _daemon_ a _no-daemon_, o vicecersa.

### _Timers_

La clase `Timer` es una subclase de la clase `Thread` y permite ejecutar un proceso o acción después que ha pasado un determinado tiempo. `Timer` requiere, como parámetros básicos, el tiempo en segundos después del cual debe ejecutarse el _thread_, el nombre del proceso a ejecutar y los argumentos de entrada para el proceso. El método `cancel()` permite, si se requiere, cancelar la ejecución del _timer_ antes que este sea ejecuctado.

In [12]:
def mi_timer(archivo):
    with open(archivo) as fid:
        for linea in fid:
            print(linea)
    fid.close()


# Los archivos para probar están junto con la materia
t1 = threading.Timer(10.0, mi_timer, args=('mensaje_01.txt',))
t2 = threading.Timer(5.0, mi_timer, args=('mensaje_02.txt',))

t1.start() # el thread t comenzará después de 10 seconds
t2.start() # el thread t comenzará después de 5 seconds

[DEBUG] (service   ) Exiting
[DEBUG] (daemon    ) Exiting
[DEBUG] (Thread-15 ) Exiting
[DEBUG] (daemon    ) Exiting


Thread 1 exiting...
Acabas de ejecutar el timer 2

Acabas de ejecutar el timer 1
